In [1]:
import os
import shutil
import random
import glob

import pandas as pd

In [2]:
IMAGE_HEIGHT = 256
IMAGE_WIDTH = 256

RESIZE_IMAGES = True

IS_DENOISING = True
IS_FREQUENCY = False

In [3]:
base_dir = './content/DataSet2/Classification'
if (IS_FREQUENCY):
    base_dir = './content/DataSet2/Freq_Classification'
elif (IS_DENOISING):
    base_dir = './content/DataSet2/Denoising'


with_noise_dir = os.path.join(base_dir,'With-Noise')
without_noise_dir = os.path.join(base_dir,'Without-Noise')

train_dir = os.path.join(base_dir, 'Train')
test_dir = os.path.join(base_dir, 'Test')
val_dir = os.path.join(base_dir, 'Validation')

## Resizing

In [4]:
from PIL import Image

def resize_images(folder_path, target_size=(IMAGE_HEIGHT, IMAGE_WIDTH)):  #resizing function
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            img = Image.open(file_path)
            print('resizing:', img)
            img_resized = img.resize(target_size, Image.LANCZOS)
            img_resized.save(file_path)

In [5]:
if (RESIZE_IMAGES):
    
    resize_images(with_noise_dir)
    resize_images(without_noise_dir)

    print("Images resized successfully!")

resizing: <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512 at 0x1C07F613D00>
resizing: <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512 at 0x1C07F6FE0D0>
resizing: <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512 at 0x1C07F5F6F70>
resizing: <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512 at 0x1C07F6FE0D0>
resizing: <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512 at 0x1C07F5F6E80>
resizing: <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512 at 0x1C07F6FE0D0>
resizing: <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512 at 0x1C07F5F6CA0>
resizing: <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512 at 0x1C07F6FE0D0>
resizing: <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512 at 0x1C07F5F6F70>
resizing: <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512 at 0x1C07F6FE0D0>
resizing: <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512 at 0x1C07F5F6E80>

## Convert to Frequency

In [6]:
import cv2
import numpy as np

def convert_to_freq(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
            freq = np.fft.fftshift(np.fft.fft2(img))
            cv2.imwrite(file_path, freq)

In [7]:
if (IS_FREQUENCY):
    
    convert_to_freq(with_noise_dir)
    convert_to_freq(without_noise_dir)

    print("Images converted successfully!")

## Class Foldeing

In [8]:
data = pd.read_excel('./content/Labels.xlsx')
print(data['Noise Type'].unique())
data.head()

['Gaussian' 'Periodic' 'Salt']


,Denoise Image,Noise Type,Noisy Image
0,Without-Noise/cast_def_0_0.jpeg,Gaussian,With-Noise/cast_def_0_0.jpeg
1,Without-Noise/cast_def_0_2.jpeg,Gaussian,With-Noise/cast_def_0_2.jpeg
2,Without-Noise/cast_def_0_22.jpeg,Gaussian,With-Noise/cast_def_0_22.jpeg
3,Without-Noise/cast_def_0_25.jpeg,Gaussian,With-Noise/cast_def_0_25.jpeg
4,Without-Noise/cast_def_0_33.jpeg,Gaussian,With-Noise/cast_def_0_33.jpeg


In [9]:
for nType in data['Noise Type'].unique():

    os.makedirs(os.path.join(with_noise_dir,nType), exist_ok=True)
    for d in data[data['Noise Type']==nType]['Noisy Image']:
        shutil.move(os.path.join(base_dir,d),os.path.join(with_noise_dir,nType))

    os.makedirs(os.path.join(without_noise_dir,nType), exist_ok=True)
    for d in data[data['Noise Type']==nType]['Denoise Image']:
        shutil.move(os.path.join(base_dir,d),os.path.join(without_noise_dir,nType))  

## Train Test Split

In [10]:
# Train Folder
os.makedirs(train_dir, exist_ok=True)
shutil.move(os.path.join(base_dir, 'With-Noise'), train_dir)
shutil.move(os.path.join(base_dir, 'Without-Noise'), train_dir)

# Test Folder
os.makedirs(os.path.join(test_dir, 'With-Noise'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'Without-Noise'), exist_ok=True)
for c in data['Noise Type'].unique():
    os.makedirs(os.path.join(test_dir+'/With-Noise', c), exist_ok=True)
    os.makedirs(os.path.join(test_dir+'/Without-Noise', c), exist_ok=True)

# Validation Folder
os.makedirs(os.path.join(val_dir, 'With-Noise'), exist_ok=True)
os.makedirs(os.path.join(val_dir, 'Without-Noise'), exist_ok=True)
for c in data['Noise Type'].unique():
    os.makedirs(os.path.join(val_dir+'/With-Noise', c), exist_ok=True)
    os.makedirs(os.path.join(val_dir+'/Without-Noise', c), exist_ok=True)

# cats
for category in ['With-Noise', 'Without-Noise']:
    src = os.path.join(train_dir, category)
    dest1 = os.path.join(test_dir, category)
    dest2 = os.path.join(val_dir, category)

    # classes
    for c in data['Noise Type'].unique():
        c_src = os.path.join(src, c)
        c_dest1 = os.path.join(dest1, c)
        c_dest2 = os.path.join(dest2, c)

    # Test 
        images = os.listdir(c_src)
        num_images_to_move = int(0.15 * len(images))  # Randomly select 15% of images to test
        images_to_move = random.sample(images, num_images_to_move)

        # Move the selected images to Test Images directory
        for image in images_to_move:
            source_path = os.path.join(c_src, image)
            dest_path = os.path.join(c_dest1, image)
            shutil.move(source_path, dest_path)

    # Val
        images = os.listdir(c_src)
        num_images_to_move = int(0.15 * len(images))  # Randomly select 15% of images to test
        images_to_move = random.sample(images, num_images_to_move)

        # Move the selected images to Test Images directory
        for image in images_to_move:
            source_path = os.path.join(c_src, image)
            dest_path = os.path.join(c_dest2, image)
            shutil.move(source_path, dest_path)

print("Images spilted successfully!")

Images spilted successfully!


In [11]:
# Must run train test split first!

if (IS_DENOISING):
    # cats
    for category in ['With-Noise', 'Without-Noise']:
        src1 = os.path.join(train_dir, category)
        src2 = os.path.join(test_dir, category)
        src3 = os.path.join(val_dir, category)

        # classes
        for c in data['Noise Type'].unique():
            c_src1 = os.path.join(src1, c)
            c_src2 = os.path.join(src2, c)
            c_src3 = os.path.join(src3, c)

            cls_dir = os.path.join(base_dir,c)

            # Train
            for image in os.listdir(c_src1):
                source = os.path.join(c_src1,image)
                dest = os.path.join(cls_dir, os.path.join('Train', category))
                if(not os.path.isdir(dest)):
                    os.makedirs(dest)
                shutil.move(source,dest)

            # Test
            for image in os.listdir(c_src2):
                source = os.path.join(c_src2,image)
                dest = os.path.join(cls_dir, os.path.join('Test', category))
                if(not os.path.isdir(dest)):
                    os.makedirs(dest)
                shutil.move(source,dest)

            # Validation
            for image in os.listdir(c_src3):
                source = os.path.join(c_src3,image)
                dest = os.path.join(cls_dir, os.path.join('Validation', category))
                if(not os.path.isdir(dest)):
                    os.makedirs(dest)
                shutil.move(source,dest)

## Check

In [12]:
def get_files_count(directory):
  if not os.path.exists(directory):
    return 0
  count=0
  for current_path,dirs,files in os.walk(directory):
    for dr in dirs:
      count+= len(glob.glob(os.path.join(current_path,dr+"/*")))
  return count

In [13]:
num_classes=len(glob.glob(train_dir+"/With-Noise/*"))

train_samples =get_files_count(train_dir)
test_samples=get_files_count(test_dir)
val_samples=get_files_count(val_dir)

print(num_classes,"Classes")
print(train_samples,"Train images")
print(test_samples,"Test images")
print(val_samples,"Val images")

3 Classes
6 Train images
6 Test images
6 Val images
